# Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# 1. Подготовка

In [0]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import time
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [0]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords as nltk_stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\poltoran\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
import torch
import transformers as ppb
from pytorch_pretrained_bert import BertTokenizer
from tqdm import notebook

Посмотрим на таблицу:

In [0]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [0]:
print('Количество строк в датафрейме:', df.shape[0])

Количество строк в датафрейме: 159571


Таблица состоит из двух столбцов, toxic - целевой признак. В столбце text приведем все сообщения к нижнему регистру.

In [0]:
df.text = df.text.str.lower()

Оценим дисбаланс классов:

In [0]:
df.toxic.value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Наблюдается сильный дисбаланс (10% класса "1" и 90% класса "0"). Это наблюдение дает нам значение accuracy для оценки адекватности моделей: accuracy моделей должна быть больше 90%.

# 2. Обучение

### 2.1. Построение моделей с применением TF-IDF 

Созданим копию датафрейма для дальнейшего использования:

In [0]:
df_tf_idf = df.copy()

Напишем функцию, которая принимает на вход неподготовленный текст, а возвращает его после очистки, токенизации, стемминга и обратной склейки в list:

In [0]:
stemmer_tf_idf = SnowballStemmer("english")
tokenizer_tf_idf = RegexpTokenizer(r'\w{2,}')

def clean_stemm(text):
    new_words = tokenizer_tf_idf.tokenize(text)
    new_list = []
    for w in new_words:
        w = stemmer_tf_idf.stem(w)
        new_list.append(w)
    new_list = ' '.join(new_list)
    return new_list

Применим функцию к датафрейму и созданим новый столбец с текстом после очистки и стемминга stem_text:

In [0]:
%%time
df_tf_idf['stem_text'] = df_tf_idf['text'].apply(clean_stemm)

Wall time: 1min 54s


Разделим датафрейм на признак и на на целевой признак, далее разделим их на обучающую и тестовую выборки:

In [0]:
%%time
corpus_tf_idf = df_tf_idf['stem_text'].values.astype('U')
target_tf_idf = df_tf_idf['toxic']

train_features_tf_idf, test_features_tf_idf, train_target_tf_idf, test_target_tf_idf = train_test_split(
corpus_tf_idf, target_tf_idf, test_size=0.2, random_state=42)

# Выведем информацию о размерах выборок, чтобы убедиться в правильной разбивке
print('Количество строк в обучающей выборке:', train_features_tf_idf.shape[0])
print('Количество строк в тестовой выборке:', test_features_tf_idf.shape[0])

Количество строк в обучающей выборке: 127656
Количество строк в тестовой выборке: 31915
Wall time: 4.67 s


Вычислим TF-IDF для корпуса текстов с учетом стоп-слов. Так как данные разделены на обучающую и тестовую выборки, функцию fit() запустим только на обучающей:

In [0]:
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf_train = count_tf_idf.fit_transform(train_features_tf_idf)
tf_idf_test = count_tf_idf.transform(test_features_tf_idf)

Данные для обучения моделей готовы. Приступаем к обучению нескольких моделей. Первой будет Логистическая регрессия. Ввиду дисбаланса классов будем использовать class_weight = 'balanced':

In [0]:
%%time

model_lr_tf_idf = LogisticRegression(random_state=12345, solver='lbfgs', class_weight = 'balanced')
model_lr_tf_idf.fit(tf_idf_train, train_target_tf_idf)
predicted_lr_tf_idf = model_lr_tf_idf.predict(tf_idf_test)
accuracy_lr_tf_idf = accuracy_score(test_target_tf_idf, predicted_lr_tf_idf)
f1_score_lr_tf_idf = f1_score(test_target_tf_idf, predicted_lr_tf_idf)

probabilities_test_lr_tf_idf = model_lr_tf_idf.predict_proba(tf_idf_test)
probabilities_one_test_lr_tf_idf = probabilities_test_lr_tf_idf[:, 1]
auc_roc_lr_tf_idf = roc_auc_score(test_target_tf_idf, probabilities_one_test_lr_tf_idf)

print('Логистическая регрессия:')
print('accuracy:', accuracy_lr_tf_idf.round(decimals=3))
print('f1_score:', f1_score_lr_tf_idf.round(decimals=3))
print('auc_roc:', auc_roc_lr_tf_idf.round(decimals=3))

Логистическая регрессия:
accuracy: 0.942
f1_score: 0.752
auc_roc: 0.973
Wall time: 3.77 s


Случайный лес:

In [0]:
max_depth_list1 = []; n_estimators_list1 = []; accuracy_list1 = []; f1_score_list1 = []; auc_roc_list1 = []

for i in range(1, 16, 1):
    model_rf_tf_idf = RandomForestClassifier(n_estimators=10, max_depth=i, random_state=12345, class_weight='balanced')
    model_rf_tf_idf.fit(tf_idf_train, train_target_tf_idf)
    predicted_rf_tf_idf = model_rf_tf_idf.predict(tf_idf_test)
    accuracy_rf_tf_idf = accuracy_score(test_target_tf_idf, predicted_rf_tf_idf)
    f1_score_rf_tf_idf = f1_score(test_target_tf_idf, predicted_rf_tf_idf)

    probabilities_test_rf_tf_idf = model_rf_tf_idf.predict_proba(tf_idf_test)
    probabilities_one_test_rf_tf_idf = probabilities_test_rf_tf_idf[:, 1]
    auc_roc_rf_tf_idf = roc_auc_score(test_target_tf_idf, probabilities_one_test_rf_tf_idf)
    
    max_depth_list1.append(i)
    n_estimators_list1.append('10')
    accuracy_list1.append(accuracy_rf_tf_idf)
    f1_score_list1.append(f1_score_rf_tf_idf)
    auc_roc_list1.append(auc_roc_rf_tf_idf)
    
top1_rf_f1_score = pd.DataFrame({'max_depth': max_depth_list1, 
                             'n_estimators':n_estimators_list1,
                             'accuracy': accuracy_list1,
                             'f1_score': f1_score_list1,
                             'auc_roc': auc_roc_list1
                             })
print(top1_rf_f1_score.sort_values('f1_score', ascending=False).head(2))

    max_depth n_estimators  accuracy  f1_score   auc_roc
14         15           10  0.621745  0.307322  0.797798
13         14           10  0.614351  0.300523  0.789242


In [0]:
max_depth_list2 = []; n_estimators_list2 = []; accuracy_list2 = []; f1_score_list2 = []; auc_roc_list2 = []

for j in range(10, 211, 20):
    model_rf_tf_idf = RandomForestClassifier(n_estimators=j, max_depth=15, random_state=12345, class_weight='balanced')
    model_rf_tf_idf.fit(tf_idf_train, train_target_tf_idf)
    predicted_rf_tf_idf = model_rf_tf_idf.predict(tf_idf_test)
    accuracy_rf_tf_idf = accuracy_score(test_target_tf_idf, predicted_rf_tf_idf)
    f1_score_rf_tf_idf = f1_score(test_target_tf_idf, predicted_rf_tf_idf)

    probabilities_test_rf_tf_idf = model_rf_tf_idf.predict_proba(tf_idf_test)
    probabilities_one_test_rf_tf_idf = probabilities_test_rf_tf_idf[:, 1]
    auc_roc_rf_tf_idf = roc_auc_score(test_target_tf_idf, probabilities_one_test_rf_tf_idf)
    
    max_depth_list2.append('15')
    n_estimators_list2.append(j)
    accuracy_list2.append(accuracy_rf_tf_idf)
    f1_score_list2.append(f1_score_rf_tf_idf)
    auc_roc_list2.append(auc_roc_rf_tf_idf)
    
top2_rf_f1_score = pd.DataFrame({'max_depth': max_depth_list2, 
                             'n_estimators':n_estimators_list2,
                             'accuracy': accuracy_list2,
                             'f1_score': f1_score_list2,
                             'auc_roc': auc_roc_list2
                             })
print(top2_rf_f1_score.sort_values('f1_score', ascending=False).head(2))

   max_depth  n_estimators  accuracy  f1_score   auc_roc
8         15           170  0.708319  0.373680  0.881702
10        15           210  0.707536  0.373389  0.880655


Результаты случайного леса плохи, скорей всего нужно применять больше деревьев. Остановимся пока на таких значениях.

Обучим модель catBoost:

In [0]:
%%time
model_cat_tf_idf = CatBoostClassifier(iterations=1000,
                                      random_seed = 42,
                                      metric_period = 50,
                                      eval_metric = 'F1',
                                      learning_rate = 0.5,
                                      early_stopping_rounds = 50, 
                                      #task_type="GPU", 
                                      #devices='0:1'
                                     )
model_cat_tf_idf.fit(tf_idf_train, train_target_tf_idf, verbose=200)
predicted_cat_tf_idf = model_cat_tf_idf.predict(tf_idf_test)
accuracy_cat_tf_idf = accuracy_score(test_target_tf_idf, predicted_cat_tf_idf)
f1_score_cat_tf_idf = f1_score(test_target_tf_idf, predicted_cat_tf_idf)

probabilities_test_cat_tf_idf = model_cat_tf_idf.predict_proba(tf_idf_test)
probabilities_one_test_cat_tf_idf = probabilities_test_cat_tf_idf[:, 1]
auc_roc_cat_tf_idf = roc_auc_score(test_target_tf_idf, probabilities_one_test_cat_tf_idf)

print('CatBoost:')
print('accuracy:', accuracy_cat_tf_idf.round(decimals=3))
print('f1_score:', f1_score_cat_tf_idf.round(decimals=3))
print('auc_roc:', auc_roc_cat_tf_idf.round(decimals=3))

0:	learn: 0.3860659	total: 1.01s	remaining: 16m 53s
200:	learn: 0.8173763	total: 2m 35s	remaining: 10m 19s
400:	learn: 0.8400955	total: 5m 11s	remaining: 7m 44s
600:	learn: 0.8524854	total: 7m 44s	remaining: 5m 8s
800:	learn: 0.8684387	total: 10m 19s	remaining: 2m 33s
999:	learn: 0.8942042	total: 12m 52s	remaining: 0us
CatBoost:
accuracy: 0.96
f1_score: 0.78
auc_roc: 0.965
Wall time: 13min 1s


Теперь обучим LightGBM на стандартных настройках гиперпараметров:

In [0]:
%%time
model_lgb_tf_idf = lgb.LGBMClassifier()
model_lgb_tf_idf.fit(tf_idf_train, train_target_tf_idf)
predicted_lgb_tf_idf = model_lgb_tf_idf.predict(tf_idf_test)
accuracy_lgb_tf_idf = accuracy_score(test_target_tf_idf, predicted_lgb_tf_idf)
f1_score_lgb_tf_idf = f1_score(test_target_tf_idf, predicted_lgb_tf_idf)

probabilities_test_lgb_tf_idf = model_lgb_tf_idf.predict_proba(tf_idf_test)
probabilities_one_test_lgb_tf_idf = probabilities_test_lgb_tf_idf[:, 1]
auc_roc_lgb_tf_idf = roc_auc_score(test_target_tf_idf, probabilities_one_test_lgb_tf_idf)

print('LightGBM:')
print('accuracy:', accuracy_lgb_tf_idf.round(decimals=3))
print('f1_score:', f1_score_lgb_tf_idf.round(decimals=3))
print('auc_roc:', auc_roc_lgb_tf_idf.round(decimals=3))

LightGBM:
accuracy: 0.959
f1_score: 0.763
auc_roc: 0.965
Wall time: 24.8 s


##### Выводы по пункту 2.1:
Три из четырех моделей превзошли необходимое значение f1>=0.75. Модель логистической регрессии обучилась быстрее бустингов (4 сек), но показала наименьший результат f1=0.752. CatBoost показал наибольшее значение  f1=0.78, но обучался дольше всех (13 мин). LightGBM - "среднячок" с f1=0.763 и временем обучения 25 сек. Более наглядное представление результатов в виде таблице приведено в конце работы.

### 2.2. Построение моделей с применением BERT

Из датафрейма получим выборку с равным количеством классов "1" и "0". Общий размер выборки примем в 2000 ввиду долгой работы модели BERT:

In [0]:
df_bert_ones = df[df['toxic']==1].sample(1000).reset_index(drop=True)
df_bert_zeros = df[df['toxic']==0].sample(1000).reset_index(drop=True)
df_bert = pd.concat([df_bert_ones] + [df_bert_zeros]).reset_index(drop=True)
df_bert = shuffle(df_bert, random_state=12345).reset_index(drop=True)
# Проверим правильность разбиения выборки по классам
df_bert.toxic.value_counts()

1    1000
0    1000
Name: toxic, dtype: int64

Создаем токенайзер для модели BERT:

In [0]:
tokenizer = ppb.BertTokenizer.from_pretrained('bert-base-uncased')

Токенизируем текст каждого твита (строки). В таблице есть строки, превыщающие длину в 512 токенов. Ограничим максимальную длину.

In [0]:
tokenized = df_bert['text'].apply((lambda x: tokenizer.encode(x, max_length=512, add_special_tokens=True)) )

Инициализируем предобученную модель BERT из файла, в json-файле конфигурации описаны параметры модели:

In [0]:
config = ppb.BertConfig.from_pretrained('bert-base-uncased')
model = ppb.BertModel.from_pretrained('bert-base-uncased', config = config)

Из-за того, что каждый твит в датасете имеет разную длину (количество токенов) мы делаем паддинг - заполнение нулями каждого массива токенов до длины максимального массива чтобы на выходе получить матрицу из токенизированных текстов одной длины:

In [0]:
len_list = tokenized.apply(lambda x: len(x))
max_len = max(len_list)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
# Накладываем маску на значимые токены
# В данном случае нам важны все слова кроме нулевых токенов, появившихся на предыдущем шаге паддинга
attention_mask = np.where(padded != 0, 1, 0)

Теперь сформируем вектора текстов с помощью модели BERT:

In [0]:
batch_size = 50
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        input_ids = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            last_hidden_states = model(input_ids, attention_mask=attention_mask_batch)

        embeddings.append(last_hidden_states[0][:,0,:].numpy())

Преобразуем список батчей эмбеддингов в numpy-матрицу, в другую матрицу запишем значения целевого признака, и разделим матрицы на обучающую и тестовую выборки.

In [0]:
features = np.concatenate(embeddings)
labels = df_bert['toxic']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, 
                                                                            test_size=0.25, random_state=42)

Все готово для обучения моделей: логистической регрессии, LightGBM и CatBoost.

In [0]:
%%time
model_lr = LogisticRegression(solver = 'lbfgs', class_weight = 'balanced', max_iter = 1000)
model_lr.fit(train_features, train_labels)
predicted_lr = model_lr.predict(test_features)
accuracy_lr = accuracy_score(test_labels, predicted_lr)
f1_score_lr = f1_score(test_labels, predicted_lr)

probabilities_test_lr = model_lr.predict_proba(test_features)
probabilities_one_test_lr = probabilities_test_lr[:, 1]
auc_roc_lr = roc_auc_score(test_labels, probabilities_one_test_lr)

print('Логистическая регрессия:')
print('accuracy:', accuracy_lr.round(decimals=3))
print('f1_score:', f1_score_lr.round(decimals=3))
print('auc_roc:', auc_roc_lr.round(decimals=3))

Логистическая регрессия:
accuracy: 0.864
f1_score: 0.865
auc_roc: 0.949
Wall time: 366 ms


In [0]:
%%time
model_lgb = lgb.LGBMClassifier()
model_lgb.fit(train_features, train_labels)
predicted_lgb = model_lgb.predict(test_features)
accuracy_lgb = accuracy_score(test_labels, predicted_lgb)
f1_score_lgb = f1_score(test_labels, predicted_lgb)

probabilities_test_lgb = model_lgb.predict_proba(test_features)
probabilities_one_test_lgb = probabilities_test_lgb[:, 1]
auc_roc_lgb = roc_auc_score(test_labels, probabilities_one_test_lgb)

print('LightGBM:')
print('accuracy:', accuracy_lgb.round(decimals=3))
print('f1_score:', f1_score_lgb.round(decimals=3))
print('auc_roc:', auc_roc_lgb.round(decimals=3))

LightGBM:
accuracy: 0.856
f1_score: 0.858
auc_roc: 0.939
Wall time: 4.43 s


In [0]:
%%time
model_cat = CatBoostClassifier(iterations=1000,
                               random_seed = 42,
                               metric_period = 50,
                               eval_metric = 'F1',
                               learning_rate = 0.5,
                               early_stopping_rounds = 50, 
                               #task_type="GPU", 
                               #devices='0:1'
                              )

model_cat.fit(train_features, train_labels, verbose=200)
predicted_cat = model_cat.predict(test_features)
accuracy_cat = accuracy_score(test_labels, predicted_cat)
f1_score_cat = f1_score(test_labels, predicted_cat)

probabilities_test_cat = model_cat.predict_proba(test_features)
probabilities_one_test_cat = probabilities_test_cat[:, 1]
auc_roc_cat = roc_auc_score(test_labels, probabilities_one_test_cat)

print('CatBoost:')
print('accuracy:', accuracy_cat.round(decimals=3))
print('f1_score:', f1_score_cat.round(decimals=3))
print('auc_roc:', auc_roc_cat.round(decimals=3))

0:	learn: 0.7500000	total: 77.8ms	remaining: 1m 17s
200:	learn: 1.0000000	total: 13.6s	remaining: 54s
400:	learn: 1.0000000	total: 27.4s	remaining: 40.9s
600:	learn: 1.0000000	total: 41.6s	remaining: 27.6s
800:	learn: 1.0000000	total: 55.5s	remaining: 13.8s
999:	learn: 1.0000000	total: 1m 9s	remaining: 0us
CatBoost:
accuracy: 0.858
f1_score: 0.862
auc_roc: 0.937
Wall time: 1min 9s


##### Выводы по пункту 2.2:
Все использованные модели сильно превысили необходимое значение метрики F1=0.75, показав результаты F1>=0.858. Также все модели сильно превысили порог адекватности в accuracy=0.5 показав результат accuracy>=0.856. Результаты в виде таблицы приведены в пунтке 3 данной работы.

# 3. Выводы

Модели с применением TF-IDF:

Метрика        | Log.Reg | CatBoost | LightGBM
:------------- | ------- | -------- | ------- 
Accuracy       | 0.942   | 0.96     | 0.959  
F1             | 0.752   | 0.78     | 0.763
ROC-AUC        | 0.973   | 0.965    | 0.965
Время обучения | 4 сек   | 13 мин   | 25 сек

Модели с применением BERT:

Метрика        | Log.Reg | CatBoost | LightGBM
:------------- | ------- | -------- | ------- 
Accuracy       | 0.864   | 0.858    | 0.856  
F1             | 0.865   | 0.862    | 0.858
ROC-AUC        | 0.949   | 0.937    | 0.939
Время обучения | <1 сек  | 1 мин    | 4 сек

- Все модели (кроме случайного леса) превысили необходимое значение метрики F=0.75.
- С применением TF-IDF лучшей моделью по значению метрики F1 является CatBoost (F1=0.78). Однако у нее самое большое время обучения - 13 мин.
- С применением BERT лучшей моделью по значению метрики F1 является логистическая регрессия (F1=0.865). У нее также самые большие значения Accuracy=0.864 и ROC-AUC=0.949 и наименьшее время обучения (<1 сек) по сравнению с CatBoost и LightGBM.